In [59]:
class Weather():
        def __init__(self, Outlook, Temp,Humidity,Windy,Play):
            self.Outlook = Outlook
            self.Temp = Temp
            self.Humidity = Humidity
            self.Windy = Windy
            self.Play = Play
            

In [60]:
def WeatherList():
    #degerler slaytdan alınmıştır
    weatherList = []
    weatherList.append(Weather("Sunny","Hot","High","F","No"))
    weatherList.append(Weather("Sunny","Hot","High","T","No"))
    weatherList.append(Weather("Overcast","Hot","High","F","Yes"))
    weatherList.append(Weather("Rainy","Mild","High","F","Yes"))
    weatherList.append(Weather("Rainy","Cool","Normal","F","Yes"))
    weatherList.append(Weather("Rainy","Cool","Normal","T","No"))
    weatherList.append(Weather("Overcast","Cool","Normal","T","Yes"))
    weatherList.append(Weather("Sunny","Mild","High","F","No"))
    weatherList.append(Weather("Sunny","Cool","Normal","F","Yes"))
    weatherList.append(Weather("Rainy","Mild","Normal","F","Yes"))
    weatherList.append(Weather("Sunny","Mild","Normal","T","Yes"))
    weatherList.append(Weather("Overcast","Mild","High","T","Yes"))
    weatherList.append(Weather("Overcast","Hot","Normal","F","Yes"))
    weatherList.append(Weather("Rainy","Mild","High","T","No"))
    return weatherList

In [61]:
def mappingForOutlook(value):
    if(value == "Sunny"):
        return 0
    elif(value == "Overcast"):
        return 1
    else:
        return 2

In [62]:
def mappingForTemp(value):
    if(value == "Hot"):
        return 0
    elif(value == "Mild"):
        return 1
    else:
        return 2

In [63]:
def mappingForHumidity(value):
    if(value == "High"):
        return 0
    else:
        return 1

In [64]:
def mappingForWindy(value):
    if(value == "T"):
        return 1
    else:
        return 0

In [65]:
def mappingForLabel(value):
    if(value == "Yes"):
        return 1
    else:
        return 0

In [66]:
def changeData():
    weatherList=WeatherList();
    data = []
    outcome = []
    for i in range(len(weatherList)):
        data.append([mappingForOutlook(weatherList[i].Outlook),
                     mappingForTemp(weatherList[i].Temp),
                     mappingForHumidity(weatherList[i].Humidity),
                     mappingForWindy(weatherList[i].Windy)])
        outcome.append(mappingForLabel(weatherList[i].Play))

    data = np.asarray(data)     
    outcome = np.asarray(outcome)
    return data,outcome

In [118]:

#Function for calculating the prior probabilities
def occurrences(outcome):
    no_of_examples = len(outcome)
    prob = dict(Counter(outcome))
    for key in prob.keys():
        if(prob[key]):
            prob[key] = prob[key] / float(no_of_examples)
        else:
            prob[key] = (prob[key]+1) / float(no_of_examples+1) 
    return prob



def naive_bayes(training, outcome, new_sample):
    classes     = np.unique(outcome)
    rows, cols  = np.shape(training)
    likelihoods = {}
    likelihoods_prob = {}
    for cls in classes:
        likelihoods[cls] = defaultdict(list)
        likelihoods_prob[cls] = defaultdict(list)
    class_probabilities = occurrences(outcome)
 
    for cls in classes:
        row_indices = np.where(outcome == cls)[0]
        subset      = training[row_indices, :]
        r, c        = np.shape(subset)
        for j in range(0,c):
            likelihoods[cls][j] += list(subset[:,j])
            #the occurrences for each feature is saved in likelihoods
 
    for cls in classes:
        for j in range(0,cols):
             likelihoods_prob[cls][j] = occurrences(likelihoods[cls][j])
             #likelihoods_prob contain the probabilities per feature per class.
 
    results = {}
    for cls in classes:
         class_probability = class_probabilities[cls]
         for i in range(0,len(new_sample)):
             relative_values = likelihoods_prob[cls][i]
             if new_sample[i] in relative_values.keys():
                 class_probability *= relative_values[new_sample[i]]
             #else:
               #  class_probability *= 0
             results[cls] = class_probability
    #print (results)
    return results
 


In [145]:
import csv
import random
import math
import numpy as np
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
random.seed() 
data,outcome = changeData()
X_train, X_test, y_train, y_test = train_test_split(data, outcome, test_size=0.30, random_state=42)
y_pred = []


for i in range(len(X_test)):
    result=naive_bayes(X_train, y_train, X_test[i])
    if(result[0]>result[1]):
        y_predic=0
    else:
        y_predic=1
    y_pred.append(y_predic)
y_pred = np.asarray(y_predict)  
confusionMatrix = confusion_matrix(y_test, y_pred)

TrueNegative=confusionMatrix[0,0]
TruePositive=confusionMatrix[1,1]
FalsePositive=confusionMatrix[0,1]
FalseNegative=confusionMatrix[1,0]

accuracy=(TruePositive+TrueNegative)/(TrueNegative+TruePositive+FalsePositive+FalseNegative)
precision=(TruePositive)/(TruePositive+FalsePositive)
specificty=(TrueNegative)/(TrueNegative+FalsePositive)
sensitivity=(TruePositive)/( TruePositive+FalseNegative)
print('Accuracy: %f' % accuracy)
print('Precision: %f' % precision)
print('Specificty: %f' % specificty)
print('Sensitivity: %f' % sensitivity)

{0: 0.024691358024691357, 1: 0.03292181069958847}
{0: 0.14814814814814814, 1: 0.008230452674897118}
{0: 0.024691358024691357, 1: 0.008230452674897118}
{0: 0.037037037037037035, 1: 0.016460905349794237}
{0: 0.07407407407407407, 1: 0.024691358024691357}
Accuracy: 0.600000
Precision: 1.000000
Specificty: 1.000000
Sensitivity: 0.333333
